In [4]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example
import tensorflow as tf

In [5]:
person_example = Example(
    features=Features(
        feature={
            "name" : Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id" : Feature(int64_list=Int64List(value=[123])),
            "emails" : Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }
    )
)

In [6]:
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
  f.write(person_example.SerializePartialToString())

In [10]:
feature_description = {
    "name" : tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id" : tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails" : tf.io.VarLenFeature(tf.string)
}

In [11]:
for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
  parsed_example = tf.io.parse_single_example(serialized_example, feature_description)

In [12]:
tf.sparse.to_dense(parsed_example["emails"], default_value=b"")

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [13]:
parsed_example["emails"].values

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [14]:
dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"]).batch(10)
for serialized_example in dataset:
  parsed_examples = tf.io.parse_example(serialized_example, feature_description)